# NASA Mars News
## Scraping

In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\trvaughn\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [4]:
# Define database and collection
db = client.mars_db
collection = db.articles

In [5]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news'
browser.visit(url)

In [31]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(browser.html, 'html.parser')
soup.prettify()

'<html lang="en">\n <head>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>\n  <title>\n   Astropedia Search Results | GUSS Astrogeology Science Center\n  </title>\n  <meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>\n  <meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>\n  <link href="css/main.css" media="screen" rel="stylesheet"/>\n  <link href="css/print.css" media="print" rel="stylesheet"/>\n  <link href="#" rel="icon" type="image/x-ico"/>\n </head>\n <body id="results">\n  <header>\n   <a href="#" style="float:right;margin-top:10px;" target="_blank">\n    <img alt="USGS: Science for a Changing World" class="logo" height="60" src="ima

In [50]:
# Retrieve the parent divs for all paragraphs
results = soup.find_all('div', class_='list_text')

# loop over results to get article data
for result in results:
    # scrape the article title
    title = result.find('div', class_='content_title').text
    
    # scrape the article paragraph
    paragraph = result.find('div', class_='article_teaser_body').text
    
    # scrape the date
    date = result.find('div', class_='list_date').text
    print(f'paragraph = {paragraph}')
    
    time.sleep(0.5)
    
    # print article data
    print('-----------------')
    print(title)
    print(paragraph)
    print(date)
   

    # Dictionary to be inserted into MongoDB
    post = {
        'title': title,
        'paragraph': paragraph,
        'date': date
    }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)

In [49]:
# Display the MongoDB records created above
articles = db.articles.find()
for article in articles:
    print(article)

{'_id': ObjectId('60676ae15dc1511a14bf3ac1'), 'title': "NASA's InSight Detects Two Sizable Quakes on Mars", 'paragraph': 'The magnitude 3.3 and 3.1 temblors originated in a region called Cerberus Fossae, further supporting the idea that this location is seismically active.', 'date': 'April  1, 2021'}
{'_id': ObjectId('60676ae15dc1511a14bf3ac2'), 'title': 'Sensors Collect Crucial Data on Mars Landings With Arrival of Perseverance ', 'paragraph': 'MEDLI2 was one of the crucial technologies onboard the rover’s protective aeroshell that helped document the entry, descent, and landing of the spacecraft. ', 'date': 'April  1, 2021'}
{'_id': ObjectId('60676ae25dc1511a14bf3ac3'), 'title': "NASA's Curiosity Mars Rover Takes Selfie With 'Mont Mercou'", 'paragraph': 'The rover also snapped a pair of panoramas to create a 3D view of the stark cliff face featured in the selfie.', 'date': 'March 30, 2021'}
{'_id': ObjectId('60676ae25dc1511a14bf3ac4'), 'title': 'NASA Ingenuity Mars Helicopter Prepare

# JPL Mars Space Images - Featured Image

In [10]:
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [11]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(browser.html, 'html.parser')
soup

<html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-toggle="collapse" type="button

In [12]:
# Declare an empty list for image url
featured_image_url = []

# Retrieve the parent div for the image
results = soup.find_all('div', class_='floating_text_area')

# Iterate through the floating text area
for result in results:
    # Use Beautiful Soup's find() method to navigate and retrieve attributes
    a = result.find('a')
    link = result.find('a')
    href = link['href']
    image_url = ('https://spaceimages-mars.com/' + href)
    print('-----------')
    print(image_url)
    featured_image_url.append(image_url)
    
    time.sleep(0.5)

    # Click the 'FULL IMAGE' button
try:
    browser.links.find_by_partial_text('FULL IMAGE').click()
          
except:
    print("Scraping Complete")

-----------
https://spaceimages-mars.com/image/featured/mars2.jpg


In [13]:
featured_image_url[0]

'https://spaceimages-mars.com/image/featured/mars2.jpg'

# Mars Facts

In [14]:
# Import dependencies
import pandas as pd

In [15]:
# Identify the url
url = 'https://galaxyfacts-mars.com/'

In [16]:
# Use Panda's 'read_html' to parse the url
tables = pd.read_html(url)
# Find the correct table
tables[1]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [17]:
# Rename the column headings
mars_df = tables[1]
mars_df.columns = ['Feature', 'Measurement']
mars_df

,Feature,Measurement
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [18]:
# Convert to an html file
mars_df.to_html('mars_data.html', classes = 'table table-striped', index = False)

# Mars Hemispheres

In [36]:
url = 'https://marshemispheres.com/'
browser.visit(url)

In [37]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(browser.html, 'html.parser')
soup

<html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href="#" style="float:

In [43]:
# for link in soup.find_all('a'):
#     print(link.get('href'))


results = soup.find_all("img", class_="thumb")
results

[<img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/>,
 <img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/>,
 <img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/>,
 <img alt="Valles Marineris Hemisphere Enhanced thumbnail" class="thumb" src="images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png"/>]

In [ ]:
# Declare and empty list for title and img_url
title = []
img_url = []

# Retrieve the parent divs for all images
results = soup.find_all('div', class_='description')

# Iterate through the thumb area
for result in results:
    # Use Beautiful Soup's find() method to navigate and retrieve attributes
    browser.links.find_by_partial_text('Cerberus Hemisphere Enhanced').click()
    a = result.find('a')
    h3 = result.find('a')
    href = link['href']
    image_url = ('https://spaceimages-mars.com/' + href)
    print('-----------')
    print(image_url)
    featured_image_url.append(image_url)
    
    time.sleep(0.5)

    # Click the 'FULL IMAGE' button
try:
    browser.links.find_by_partial_text('FULL IMAGE').click()
          
except:
    print("Scraping Complete")

# # loop over results to get image
# for result in results:
#     # scrape Valles image
#     valles = result.find('a', src = 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg').text
    
#     # scrape Cerberus image
#     cerberus = result.find('a', src ='https://marshemispheres.com/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png').text
    
#     # scrape Cerberus image
#     schiaparelli = result.find('a', src = 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg').text
                           
#     # scrape Cerberus image
#     syrtis = result.find('a', src = 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg').text
    
#     time.sleep(0.5)
    
#     # print article data
#     print('-----------------')
#     print(title)
#     print(img_url)
   
    
#     hemisphere_image_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg"},
#     {"title": "Cerberus Hemisphere", "img_url": "https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg"},
#     {"title": "Schiaparelli Hemisphere", "img_url": "https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg"},
#     {"title": "Syrtis Major Hemisphere", "img_url": "https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg"},
# ]
    
#     # Insert dictionary into MongoDB as a document
#     collection.insert_one(post)
    
#    

In [ ]:
browser.quit()